## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-24-05-48-00 +0000,wsj,"Lithium Giant Cheers U.S., Australia Minerals ...",https://www.wsj.com/business/lithium-giant-che...
1,2025-10-24-05-44-18 +0000,nyt,Trump Says He’s Cutting Off Trade Negotiations...,https://www.nytimes.com/2025/10/23/us/politics...
2,2025-10-24-05-34-26 +0000,bbc,NBA stars and mafia among dozens arrested in i...,https://www.bbc.com/news/articles/cvgmpp3x5x5o...
3,2025-10-24-05-27-00 +0000,wsj,Full Impact of Tariffs on Asia-Pacific Still t...,https://www.wsj.com/economy/full-impact-of-tar...
4,2025-10-24-05-16-27 +0000,bbc,'Getting financial help through my GP has impr...,https://www.bbc.com/news/articles/c0rpq9y2gq0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,68
333,new,18
12,off,14
217,russian,14
216,sanctions,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
299,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...,134
150,2025-10-23-20-25-31 +0000,nyt,Trump’s Sanctions on Russian Oil Sector Ratche...,https://www.nytimes.com/2025/10/23/us/politics...,132
180,2025-10-23-18-09-53 +0000,nypost,"China, India curb Russian oil purchases after ...",https://nypost.com/2025/10/23/world-news/china...,127
276,2025-10-23-13-32-00 +0000,nyt,Trump Imposes Sanctions on Russian Oil Compani...,https://www.nytimes.com/2025/10/22/us/politics...,126
27,2025-10-24-03-21-49 +0000,cbc,Trump says he's terminating trade talks with C...,https://www.cbc.ca/news/world/trump-trade-nego...,124


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
299,134,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...
30,55,2025-10-24-03-00-00 +0000,wsj,Will India and China Be Able to Resist U.S. Sa...,https://www.wsj.com/world/asia/india-china-tru...
126,54,2025-10-23-21-24-00 +0000,wsj,Lawmakers failed to provide a last-minute repr...,https://www.wsj.com/politics/policy/air-traffi...
124,44,2025-10-23-21-29-34 +0000,latimes,"Trump calls off San Francisco 'surge,' but Eas...",https://www.latimes.com/california/story/2025-...
40,44,2025-10-24-01-40-54 +0000,nypost,White House official suggests married ex-GOP R...,https://nypost.com/2025/10/23/us-news/white-ho...
274,43,2025-10-23-13-34-07 +0000,nypost,"King Charles, Pope Leo pray together in rare m...",https://nypost.com/2025/10/23/world-news/king-...
99,42,2025-10-23-22-21-54 +0000,nypost,How mafiosos and NBA stars used rigged card sh...,https://nypost.com/2025/10/23/us-news/how-mafi...
232,41,2025-10-23-15-38-53 +0000,nypost,Trump pardons convicted Binance founder Changp...,https://nypost.com/2025/10/23/business/trump-p...
28,41,2025-10-24-03-16-11 +0000,nypost,Trump terminates trade talks with Canada over ...,https://nypost.com/2025/10/23/us-news/trump-te...
187,40,2025-10-23-18-06-16 +0000,nypost,French museum was robbed of 2K gold and silver...,https://nypost.com/2025/10/23/world-news/a-sec...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
